---
# Demo Use Case - Ingest Live Video Streams (Part 1)

A data scientist is part of a team developing an advanced driver-assistance system (ADAS).
They continuously collect video, lidar, and other sensor data from their fleet of test vehicles.
The data scientist wants to test a new object detection model on video collected today.

The purpose of this Jupyter notebook is to simulate the ingestion of real-time video into Streaming Data Platform (SDP).

To avoid the need for a set of live cameras for this demo, we play back video from a series of PNG files on disk
and write each video frame to SDP.
These videos are part of the the [KITTI Vision Benchmark Suite](http://www.cvlibs.net/datasets/kitti/raw_data.php).

- We simulate up to 8 simultaneous cameras from different vehicles.
- Each video frame is 1242 x 374 pixels, RGB color, PNG format.
- PNG image sizes average 821 KB with the largest at 964 KB.
- Each camera records at 0.5 frames per second.
- The total ingest rate is 4.3 MB/sec.

---

### Prerequisites

1. [install_dependencies.ipynb](install_dependencies.ipynb).
2. [download_kitti.ipynb](download_kitti.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl
import os

### Define Pravega stream parameters

In [3]:
#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
#gateway = '10.246.27.131:54672'
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
#stream = 'object-detector-input-video'
stream = 'kitti-1'

### Initialize connection to Pravega GRPC Gateway

In [4]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [5]:
#pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=scope))

### Create Pravega stream

In [6]:
output_stream = OutputStream(pravega_client, scope, stream)
#output_stream.delete_stream()
output_stream.create_stream()

In [7]:
#output_stream.truncate_stream()

### Ingest JPEG images from files (multiple cameras)

In [10]:
prefix = '../../data/kitti/'
camera_filespecs = [
    prefix + '2011_09_26/2011_09_26_drive_0005_sync/image_02/data/*.jpg',
    prefix + '2011_09_26/2011_09_26_drive_0009_sync/image_02/data/*.jpg',
#     prefix + '2011_09_26/2011_09_26_drive_0011_sync/image_02/data/*.jpg',
#     prefix + '2011_09_26/2011_09_26_drive_0051_sync/image_02/data/*.jpg',
#     '../../../kitti/2011_09_26/2011_09_26_drive_0009_sync/image_02/data/*.png',
#     '../../../kitti/2011_09_26/2011_09_26_drive_0011_sync/image_02/data/*.png',
#     '../../../kitti/2011_09_26/2011_09_26_drive_0014_sync/image_02/data/*.png',
#     '../../../kitti/2011_09_26/2011_09_26_drive_0051_sync/image_02/data/*.png',
#     '../../../kitti/2011_09_26/2011_09_26_drive_0059_sync/image_02/data/*.png',
#     '../../../kitti/2011_09_26/2011_09_26_drive_0084_sync/image_02/data/*.png',
#     '../../../kitti/2011_09_29/2011_09_29_drive_0071_sync/image_02/data/*.png',
]
fps = 10

In [23]:
# imp.reload(video);
# import pravega.video as video
# from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer, ImageFileSequenceLoader
# imp.reload(video);

In [24]:
loader = ImageFileSequenceLoader(scope, stream, camera_filespecs, fps)
events_to_write = loader.event_generator()
output_stream.write_events(events_to_write)